## Scrapping Winter Proton

#### Importing Packages

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime 

#### Scrapping

In [5]:
chrome_path=r"chromedriver.exe"

link = "https://www.cdiscount.com/informatique/ordinateurs-pc-portables/pc-portables/lf-228394_217-%5B1100,1190%5D.html#_his_"
driver = webdriver.Chrome(chrome_path)
driver.get(link)
time.sleep(15) #Let the page load and avoid having captcha ruin our scrapping

data = {'title' : [], 'price' : [], 'seller' : [], 'page' : [], 'caracteristics' : [], 'rating' : [], 'state' : [], 'timestamps' : [], 'link' : []}
df = pd.DataFrame(data) #Initializing the dataframe
is_not_last_page = True
page = 0
while(is_not_last_page) :
    
    time.sleep(3) #wait 3 seconds so the page has time to load
    html=driver.page_source
    titre = [] #Setting up the lists for the current page
    prix = []
    vendeur = []
    cara = []
    ratings = []
    states = []
    elements = driver.find_elements_by_css_selector('.prdtBlocInline.jsPrdtBlocInline') #Making a list of all the items sold on the page
    for element in elements : #Taking the information we need for each item of the list
        try :
            title = element.find_element_by_class_name("prdtBILTit")
            titre.append(title.text)
        except NoSuchElementException: #try and except to make sure even if there is no title (or other information) the code doesn't break
            titre.append(None)
        
        try :
            price = element.find_element_by_css_selector('.hideFromPro.priceColor.price')
            number_price = list(price.text) #Getting rid of the € sign in the price and making it a float
            number_price[-3] = '.'
            number_price = float(''.join(number_price))
            prix.append(number_price)
        except NoSuchElementException:
            prix.append(None)
        
        try :
            seller = element.find_element_by_class_name('prdtBILSeller')
            seller_text = seller.text
            begin = seller_text.find('par')
            end = seller_text.find('et') #Getting the name of the seller (there are multiple ways it can be displayed hence the tests)
            if end<begin :
                name_seller = seller_text[begin + 4:len(seller_text)]
            else :
                name_seller = seller_text[begin + 4:end]
            vendeur.append(name_seller)
        except NoSuchElementException:
            vendeur.append(None)
        try:
            caracteristics = element.find_element_by_class_name('prdtBILDesc')
            cara.append(caracteristics.text)
        except NoSuchElementException:
            cara.append(None)
        try:
            rating = element.find_element_by_class_name('onS')
            rating = rating.value_of_css_property('clip')
            rating = rating[10:]
            end = rating.find('px') #The display of the rating is complicated, they use the size of a png to make more or less stars appear, so we get the number of pixels out of 70 as the rating
            rating = int(rating[:end])
            ratings.append(rating)
        except NoSuchElementException:
            ratings.append(None)
        
        try :
            state = element.find_element_by_class_name('prdtBILState')
            states.append(state.text)
        except NoSuchElementException:
            states.append(None)
            
        
    pages = [page for k in range(len(elements))] #putting the page number for each article
    t = datetime.now() #getting a timestamp for the page
    timestamp = [t for k in range(len(elements))]
    links =  [link for k in range(len(elements))] #the link of the page (might be the same for most articles)
    data = {'title' : titre, 'price' : prix, 'seller' : vendeur, 'page' : page, 'caracteristics' : cara, 'rating' : ratings, 'state' : states, 'timestamps' : timestamp, 'link' : links}
    df = df.append(pd.DataFrame(data), ignore_index = True) #adding the new informations scrapped on the page to the global dataframe
    page += 1
    try :
        next_page = driver.find_element_by_link_text('Page suivante')
        next_page.click()
    except NoSuchElementException:
        is_not_last_page = False #If the current page is the last, catches the error and ends the loop
    print(page) #just to make sure the program is running at a normal pace and there is no problem
 
    
driver.quit()


<ipython-input-5-f1781481eeea>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)
<ipython-input-5-f1781481eeea>:22: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  elements = driver.find_elements_by_css_selector('.prdtBlocInline.jsPrdtBlocInline')
C:\Users\jules\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:446: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
C:\Users\jules\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:484: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
<ipython-input-5-f1781481eeea>:81: DeprecationWarning: find_element_

2
3


KeyboardInterrupt: 

#### Display of the dataframe

In [10]:
df

,title,price,seller,page,caracteristics,rating,state,timestamps,link
0,ALLDOCUBE iWork 20 - 2 en 1 PC Portable 10 pou...,279.00,Alldocube FR,0.0,Usage : Basique & Bureautique\nRAM : 4Go\nRéso...,67.0,,2022-01-06 20:32:05.157045,https://www.cdiscount.com/informatique/ordinat...
1,Ordinateur Portable 10.1 Pouces Windows 10 Net...,229.00,QCY,0.0,RAM : 3 Go\nStockage principal : 32 GB SSD\nPr...,63.0,,2022-01-06 20:32:05.157045,https://www.cdiscount.com/informatique/ordinat...
2,Ordinateur Portable 10.1 Pouces Windows 10 Net...,234.50,QCY,0.0,RAM : 3 Go\nStockage principal : 32 GB SSD\nPr...,70.0,,2022-01-06 20:32:05.157045,https://www.cdiscount.com/informatique/ordinat...
3,"Pack MICROSOFT - Surface Go 2 - 10,5"" - RAM 4G...",601.18,Cdiscount,0.0,RAM : 4 Go\nRésolution : 1920 x 1080 (Full HD)...,NaN,,2022-01-06 20:32:05.157045,https://www.cdiscount.com/informatique/ordinat...
4,Ordinateur Portable 10.1 Pouces Windows 10 Net...,229.00,QCY,0.0,RAM : 3 Go\nStockage principal : 32 GB SSD\nPr...,NaN,,2022-01-06 20:32:05.157045,https://www.cdiscount.com/informatique/ordinat...
...,...,...,...,...,...,...,...,...,...
111,"10,1 pouces a64 wifi android 7.1 ordinateur po...",107.09,ensuite,2.0,"Emballage inclus: 1 set * ordinateur portable,...",None,None,2022-01-06 20:32:22.680441,https://www.cdiscount.com/informatique/ordinat...
112,"2+16G ordinateur 10,1 pouces a64 wifi android ...",102.37,ensuite,2.0,"Emballage inclus: 1 set * ordinateur portable,...",None,None,2022-01-06 20:32:22.680441,https://www.cdiscount.com/informatique/ordinat...
113,10 pouces ordianateur protable Android 5.1 Qua...,103.09,ensuite,2.0,"""Caractéristiques: Fréquence du processeur: 1,...",None,None,2022-01-06 20:32:22.680441,https://www.cdiscount.com/informatique/ordinat...
114,Ordinateur portable Android de réseau WiFi S50...,94.24,ensuite,2.0,Emballage inclus: Ordinateur portable * 1Adapt...,None,None,2022-01-06 20:32:22.680441,https://www.cdiscount.com/informatique/ordinat...
